## 导入库

In [4]:
import pickle, sys, gc
import pandas as pd
import numpy as np
sys.path.append("./common")
sys.path.append("./dongdong/utils")
model_save_dir = "/home/kesci/work/common/models/"
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, \
ORI_TEST_NAMES, AnalysisCSV, ORI_TRAIN_NAMES, ReduceMemUsage, AnalysisHDF, ReadCSV1

## 构造数据集

In [11]:
! ls -alh /home/kesci/input/bytedance/train_final.csv
! ls -alh /home/kesci/input/bytedance/test_final_part1.csv

-rwxrwxrwx 1 kesci 1000 82G Jun 20 06:41 /home/kesci/input/bytedance/train_final.csv
-rwxrwxrwx 1 kesci 1000 1.7G Jun 20 06:41 /home/kesci/input/bytedance/test_final_part1.csv


In [10]:
# # 取最后一亿条数据作为训练集
!head -n 50000000 /home/kesci/input/bytedance/train_final.csv > /home/kesci/work/inputs/train_first_5000w.csv

In [8]:
!tail -n 5000000 /home/kesci/work/inputs/train_first_5000w.csv > /home/kesci/work/inputs/train_flast_500w.csv

In [10]:
! tail -n 1000000 /home/kesci/work/inputs/train_flast_500w.csv > /home/kesci/work/inputs/train_flast_100w.csv

In [11]:
! head -n 4000000 /home/kesci/work/inputs/train_flast_500w.csv > /home/kesci/work/inputs/train_flast_400w.csv

In [17]:
! ls -alh /home/kesci/work/inputs

total 11G
drwxr-xr-x 2 kesci root  4.0K Jul 31 08:37 .
drwxr-xr-x 9 kesci root  4.0K Jul 29 14:20 ..
-rw-r--r-- 1 kesci users 4.1G Jul 31 08:33 train_first_5000w.csv
-rw-r--r-- 1 kesci users  83M Jul 31 08:37 train_flast_100w.csv
-rw-r--r-- 1 kesci users 116M Jul 31 08:34 train_flast_140w.csv
-rw-r--r-- 1 kesci users  33M Jul 31 08:36 train_flast_40w.csv
-rw-r--r-- 1 kesci users  84M Jul 18 12:06 train_last_100w.csv
-rw-r--r-- 1 kesci users 4.1G Jul 20 02:59 train_last_5000w.csv
-rw-r--r-- 1 kesci users 4.2M Jul 23 15:39 train_last_5w.csv
-rw-r--r-- 1 kesci users 2.0G Jul 31 08:37 word2vec_sentences_last_5000w_withTest.txt


In [26]:
! head -n 20 /home/kesci/work/inputs/test_final_part1.csv

0,1,1427 28 7825 602 1427,1,4767 36 29168 13423 15 3854 9206 28 27 602 7825 7722 513 1907 11
1,1,1427 28 7825 602 1427,2,28 602 130 7825
2,1,1427 28 7825 602 1427,3,308 28 602 7825 3542 557 27 52 175
3,1,1427 28 7825 602 1427,4,945 821 28 1539 27 7825 18744 107 131 146 22326
4,1,1427 28 7825 602 1427,5,602 7825 28 27 11483 15 855 38 257
5,1,1427 28 7825 602 1427,6,602 7825 1346 16780 15 28 29057 17505
6,1,1427 28 7825 602 1427,7,602 11676 15 7825 28
7,2,121 24914 745 120,1,24914 21 9652 21 120 15 53 27 28966 352 27 41722 71202 27 3576 5 107 490 1000 40 52
8,2,121 24914 745 120,2,24914 168861 111 2882 37 395 120 4 3627 15 8302 11472 22078 27 1339 47 22 745
9,2,121 24914 745 120,3,9510 24914 7 1130 9469 27 81 7 107 24639 120 27 6710 3922 2432 11452 59 4826
10,2,121 24914 745 120,4,1665 24914 15 583 53 27 3530 4473 2905 27 7174 120 40143 33129 11
11,3,3228 125 1899 34710,1,47640 1899 2174 456 8626 27 621 4027 1733 27 303 4048 2283 290 427 11
12,3,3228 125 1899 34710,2,47640 1899 2174 27 2

In [28]:
! head -n 20 /home/kesci/work/inputs/test_final_part1.csv

,query_id,query,query_title_id,title
0,1,1427 28 7825 602 1427,1,4767 36 29168 13423 15 3854 9206 28 27 602 7825 7722 513 1907 11
1,1,1427 28 7825 602 1427,2,28 602 130 7825
2,1,1427 28 7825 602 1427,3,308 28 602 7825 3542 557 27 52 175
3,1,1427 28 7825 602 1427,4,945 821 28 1539 27 7825 18744 107 131 146 22326
4,1,1427 28 7825 602 1427,5,602 7825 28 27 11483 15 855 38 257
5,1,1427 28 7825 602 1427,6,602 7825 1346 16780 15 28 29057 17505
6,1,1427 28 7825 602 1427,7,602 11676 15 7825 28
7,2,121 24914 745 120,1,24914 21 9652 21 120 15 53 27 28966 352 27 41722 71202 27 3576 5 107 490 1000 40 52
8,2,121 24914 745 120,2,24914 168861 111 2882 37 395 120 4 3627 15 8302 11472 22078 27 1339 47 22 745
9,2,121 24914 745 120,3,9510 24914 7 1130 9469 27 81 7 107 24639 120 27 6710 3922 2432 11452 59 4826
10,2,121 24914 745 120,4,1665 24914 15 583 53 27 3530 4473 2905 27 7174 120 40143 33129 11
11,3,3228 125 1899 34710,1,47640 1899 2174 456 8626 27 621 4027 1733 27 303 4048 2283 290 427 11
12,3,3228 

In [29]:
tmp = ReadCSV("/home/kesci/work/inputs/test_final_part1.csv", names=ORI_TEST_NAMES, iterator=False)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [30]:
print(tmp[:10])

       query_id                  query  query_title_id  \
NaN    query_id                  query  query_title_id   
 0.0          1  1427 28 7825 602 1427               1   
 1.0          1  1427 28 7825 602 1427               2   
 2.0          1  1427 28 7825 602 1427               3   
 3.0          1  1427 28 7825 602 1427               4   
 4.0          1  1427 28 7825 602 1427               5   
 5.0          1  1427 28 7825 602 1427               6   
 6.0          1  1427 28 7825 602 1427               7   
 7.0          2      121 24914 745 120               1   
 8.0          2      121 24914 745 120               2   
 9.0          2      121 24914 745 120               3   
 10.0         2      121 24914 745 120               4   

                                                   title  
NaN                                                title  
 0.0   4767 36 29168 13423 15 3854 9206 28 27 602 782...  
 1.0                                     28 602 130 7825  
 2.0     

In [32]:
! head -n 20 /home/kesci/work/inputs/test_final_part1.csv

1,1427 28 7825 602 1427,1,4767 36 29168 13423 15 3854 9206 28 27 602 7825 7722 513 1907 11
1,1427 28 7825 602 1427,2,28 602 130 7825
1,1427 28 7825 602 1427,3,308 28 602 7825 3542 557 27 52 175
1,1427 28 7825 602 1427,4,945 821 28 1539 27 7825 18744 107 131 146 22326
1,1427 28 7825 602 1427,5,602 7825 28 27 11483 15 855 38 257
1,1427 28 7825 602 1427,6,602 7825 1346 16780 15 28 29057 17505
1,1427 28 7825 602 1427,7,602 11676 15 7825 28
2,121 24914 745 120,1,24914 21 9652 21 120 15 53 27 28966 352 27 41722 71202 27 3576 5 107 490 1000 40 52
2,121 24914 745 120,2,24914 168861 111 2882 37 395 120 4 3627 15 8302 11472 22078 27 1339 47 22 745
2,121 24914 745 120,3,9510 24914 7 1130 9469 27 81 7 107 24639 120 27 6710 3922 2432 11452 59 4826
2,121 24914 745 120,4,1665 24914 15 583 53 27 3530 4473 2905 27 7174 120 40143 33129 11
3,3228 125 1899 34710,1,47640 1899 2174 456 8626 27 621 4027 1733 27 303 4048 2283 290 427 11
3,3228 125 1899 34710,2,47640 1899 2174 27 2640 14 1007 446 27 811 6058 1

In [27]:
AnalysisCSV("/home/kesci/input/bytedance/test_final_part1.csv", print_rows=100)

--->file shape:
(19999999, 4)
--->100 rows of file:
     1                       1427 28 7825 602 1427  1.1  \
0    1                       1427 28 7825 602 1427    2   
1    1                       1427 28 7825 602 1427    3   
2    1                       1427 28 7825 602 1427    4   
3    1                       1427 28 7825 602 1427    5   
4    1                       1427 28 7825 602 1427    6   
5    1                       1427 28 7825 602 1427    7   
6    2                           121 24914 745 120    1   
7    2                           121 24914 745 120    2   
8    2                           121 24914 745 120    3   
9    2                           121 24914 745 120    4   
10   3                         3228 125 1899 34710    1   
11   3                         3228 125 1899 34710    2   
12   3                         3228 125 1899 34710    3   
13   4                             17022 9777 1998    1   
14   4                             17022 9777 1998    2   
15  

In [5]:
debug_feature = np.load("/home/kesci/work/common/features/nd_debugConcatFeatures.npy")

In [6]:
print(debug_feature[:10])

[[2.60000000e+01 5.30000000e+01 4.90478516e-01 2.03906250e+00
  5.30000000e+01 2.60000000e+01 2.70000000e+01 3.95000000e+01
  5.00000000e+00 1.30000000e+01 3.84521484e-01 2.59960938e+00
  1.30000000e+01 5.00000000e+00 8.00000000e+00 9.00000000e+00
  1.11083984e-01 5.88378906e-02 2.73681641e-01 2.91259766e-01
  0.00000000e+00 0.00000000e+00 2.31567383e-01 2.44262695e-01
  3.70000000e+01 6.00097656e-01 7.99804688e-01 4.10000000e+01
  5.00000000e+01 3.80000000e+01 5.40000000e+01 4.10000000e+01
  2.50195312e+00 5.50000000e+00 4.55078125e-01 2.19726562e+00
  2.99609375e+00 5.50000000e+00 2.50195312e+00 4.00000000e+00
  9.67285156e-01 1.39062500e+00 7.74218750e+00 3.19335938e-01
  1.82500000e+01 2.08000000e+02 2.18261719e-01 8.00000000e+00
  9.90234375e-01 1.40722656e+00 8.80468750e+00 3.41015625e+00
  5.56640625e+00 6.12304688e-01 1.63281250e+00 2.15820312e+00
  5.56640625e+00 3.41015625e+00 4.48828125e+00]
 [2.60000000e+01 5.60000000e+01 4.64355469e-01 2.15429688e+00
  5.60000000e+01 2.600

In [4]:
!ls -alh /home/kesci/work/common/

total 72K
drwxr-xr-x 6 kesci root  4.0K Jul 31 16:21 .
drwxr-xr-x 9 kesci root  4.0K Jul 29 14:20 ..
-rw-r--r-- 1 kesci root   323 Jul 25 15:38 config.py
drwxr-xr-x 6 kesci root  4.0K Jul 31 12:06 features
-rw-r--r-- 1 kesci root  2.5K Jul 31 16:47 models.py
drwxr-xr-x 2 kesci users 4.0K Aug  1 08:42 __pycache__
-rw-r--r-- 1 kesci root  8.7K Jul 31 11:11 run_hyperopt1.py
-rw-r--r-- 1 kesci root  7.6K Jul 31 16:47 run_hyperopt.py
-rw-r--r-- 1 kesci root  7.9K Aug  1 08:42 run_hyperopt_xp.py
drwxr-xr-x 2 kesci root  4.0K Jul 31 23:25 util_models
-rw-r--r-- 1 kesci root   11K Aug  1 05:33 utils.py
drwxr-xr-x 2 kesci root  4.0K Jul 31 11:22 util_train


In [2]:
! cd  /home/kesci/work/outputs; mkdir test

## 缺包看我

In [3]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple python-Levenshtein
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple hyperopt
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple fuzzywuzzy
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple modin
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple simhash
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple learning2rank

    100% |████████████████████████████████| 122kB 505kB/s ta 0:00:01
    100% |████████████████████████████████| 419kB 2.6MB/s eta 0:00:01


## 多进程，快！

In [10]:
import multiprocessing
import pandas as pd
import numpy as np
from multiprocessing import Pool
import scipy.sparse as sp

num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-10 # I like to leave some cores for other processes
print(num_partitions)

6


In [64]:
def parallelize_map2(func, df1, df2, num_partitions):
    # df1 and df2 must be a pandas row
    a = np.array_split(df1, num_partitions)
    b = np.array_split(df2, num_partitions)
    c = zip(a,b)
    del df1, df2, a, b
    pool = Pool(num_cores)
    tmp = pool.starmap(func, c)
    res = []
    for _ in tmp:
        res.extend(_)
    print(res)
    pool.close()
    pool.join()
    return df

In [59]:
def parallelize_map1(func, df, num_partitions):
    # df must be a pandas row
    arr = np.array_split(df, num_partitions)
    # print("input:", arr)
    pool = Pool(num_cores)
    tmp = pool.map(func, arr)
    # print("res1:", tmp)
    res = []
    for _ in tmp:
        res.extend(_)
    print("res2", res)
    pool.close()
    pool.join()
    return res

In [62]:
df = pd.DataFrame({'col': ["i like you", "she hate you", "ha ha", "ha ha uo", "ha ha me"]})
def k(x):
    return len(x)
def test(arr):
    return list(map(lambda x: x.split(), arr))
new_df = parallelize_map1(test, df["col"].values, 2)
def test(arr):
    return list(map(lambda x: k(x), arr))
new_df1 = parallelize_map1(test, new_df, 2)

res2 [['i', 'like', 'you'], ['she', 'hate', 'you'], ['ha', 'ha'], ['ha', 'ha', 'uo'], ['ha', 'ha', 'me']]
res2 [3, 3, 2, 3, 3]


In [65]:
df1 = pd.DataFrame({'col': [0,1,2,3,4,5,6,7,8,9]})
df2 = pd.DataFrame({'col': [0,1,2,3,4,5,6,7,8,9]})
def test(x, y):
    return x + y
new_df = parallelize_map2(test, df1["col"].values, df2["col"].values, 2)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [66]:
a = [1, 2, 3]
print(type(a) in [list])

True


In [74]:
# AnalysisCSV("/home/kesci/work/common/features/textMining/debug_trainTextMining_len.csv.gz")

In [75]:
# AnalysisCSV("/home/kesci/work/common/features/textMining/debug_trainTextMining_lenNoMulti.csv.gz")

## hdf5使用

In [6]:
df = pd.DataFrame({'A': [1, 2, 3]*50000000, 'B': [4, 5, 6]*50000000})

In [15]:
with Timer("test hdf5"):
    df.to_hdf('data.h5', key='df', mode='w')
# with Timer("test csv"):
#     df.to_csv("tmp", index=None, mode="a", header=True, compression='gzip', chunksize=256*1024*1024)

----->started 'test hdf5' block...
----->finished 'test hdf5' block, time used:6.36s.


In [8]:
reread = pd.read_hdf('data.h5')
print(reread[:10])

   A  B
0  1  4
1  2  5
2  3  6
3  1  4
4  2  5
5  3  6
6  1  4
7  2  5
8  3  6
9  1  4


In [11]:
reread = pd.read_csv('tmp.csv.gz')
print(reread[:10])

   A  B
0  1  4
1  2  5
2  3  6
3  1  4
4  2  5
5  3  6
6  1  4
7  2  5
8  3  6
9  1  4


In [12]:
del reread
gc.collect()

65

## Ps

In [2]:
import time
from IPython.display import clear_output
import os
while True:
    !ps aux
    time.sleep(10)
    clear_output()

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
kesci         1  0.0  0.0 353948 53340 ?        Ssl  Jul19   0:10 /opt/conda/bin
kesci         6  0.0  0.0 187136  1456 ?        Sl   Jul19   0:00 gotty --port 8
kesci       120  0.0  0.0  44620  6876 ?        S    Jul19   0:00 /opt/conda/bin
kesci       121  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       460  0.0  7.2 10614456 9138864 ?    S    Jul19   0:00 python -m ipyk
kesci       461  0.0  7.2 10614456 9138868 ?    S    Jul19   0:00 python -m ipyk
kesci       462  0.0  7.2 10614456 9138852 ?    S    Jul19   0:00 python -m ipyk
kesci       463  0.0  7.2 10614712 9138864 ?    S    Jul19   0:00 python -m ipyk
kesci       464  0.0  7.2 10614968 9139268 ?    S    Jul19   0:00 python -m ipyk
kesci       465  0.0  7.2 10614968 9139284 ?    S    Jul19   0:00 python -m ipyk
kesci       466  0.0  7.2 10614968 9139292 ?    S    Jul19   0:00 python -m ipyk
kesci       467  0.0  7.2 10614968 

KeyboardInterrupt: 

In [3]:
! ps aux
!kill -9 1028

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
kesci         1  0.0  0.0 354092 53636 ?        Ssl  Jul19   0:10 /opt/conda/bin
kesci         6  0.0  0.0 187136  1456 ?        Sl   Jul19   0:00 gotty --port 8
kesci       120  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       121  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       460  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       461  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       462  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       463  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       464  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       465  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       466  0.0  0.0      0     0 ?        Z    Jul19   0:00 [python] <defu
kesci       467  0.0  0.0      0   

## 杂项

In [52]:
!ls -alh /home/kesci/work/inputs

total 4.2G
drwxr-xr-x 2 kesci root  4.0K Jul 20 14:43 .
drwxr-xr-x 8 kesci root  4.0K Jul 21 03:02 ..
-rw-r--r-- 1 kesci users  84M Jul 18 12:06 train_last_100w.csv
-rw-r--r-- 1 kesci users 4.1G Jul 20 02:59 train_last_5000w.csv


In [74]:
! ls -alh /home/kesci/work/common/features/textMining/

total 4.8G
drwxr-xr-x 2 kesci root  4.0K Jul 22 09:26 .
drwxr-xr-x 6 kesci root  4.0K Jul 22 01:47 ..
-rw-r--r-- 1 kesci users  15M Jul 21 07:47 debug_trainTextMining_editSim.h5
-rw-r--r-- 1 kesci users 724K Jul 20 06:54 debug_trainTextMining_hashSim.h5
-rw-r--r-- 1 kesci users  23M Jul 21 07:44 debug_trainTextMining_len.h5
-rw-r--r-- 1 kesci users 6.0M Jul 22 09:26 debug_trainTextMining_len_xp.h5
-rw-r--r-- 1 kesci users  37M Jul 21 07:49 debug_trainTextMining_ngramSim.h5
-rw-r--r-- 1 kesci users 292M Jul 21 12:26 online_testTextMining_editSim.h5
-rw-r--r-- 1 kesci users 397M Jul 20 05:34 online_testTextMining_len.h5
-rw-r--r-- 1 kesci users 648M Jul 21 13:24 online_testTextMining_ngramSim.h5
-rw-r--r-- 1 kesci users 726M Jul 21 11:29 online_trainTextMining_editSim.h5
-rw-r--r-- 1 kesci users 991M Jul 20 05:21 online_trainTextMining_len.h5
-rw-r--r-- 1 kesci users 1.8G Jul 21 12:23 online_trainTextMining_ngramSim.h5


In [75]:
! ls -alh /home/kesci/work/common/features/vectorSpace/

total 4.8G
drwxr-xr-x 2 kesci root  4.0K Jul 21 23:29 .
drwxr-xr-x 6 kesci root  4.0K Jul 22 01:47 ..
-rw-r--r-- 1 kesci users  52M Jul 21 08:02 debug_trainVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users  20M Jul 22 03:09 debug_trainVectorSpace_tfidfVecSim_xp.h5
-rw-r--r-- 1 kesci users 980M Jul 21 17:39 online_testVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users 373M Jul 21 14:01 online_testVectorSpace_tfidfVecSim.h5
-rw-r--r-- 1 kesci users 2.5G Jul 21 15:27 online_trainVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users 933M Jul 21 17:05 online_trainVectorSpace_tfidfVecSim.h5


In [55]:
! ls -alh /home/kesci/work/common/features/word2vec/

total 923M
drwxr-xr-x 2 kesci root  4.0K Jul 21 23:21 .
drwxr-xr-x 6 kesci root  4.0K Jul 21 04:52 ..
-rw-r--r-- 1 kesci users  14M Jul 21 08:07 debug_trainWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 264M Jul 21 23:24 online_testWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 646M Jul 21 21:44 online_trainWord2vec_avgSenVecSim.h5


In [79]:
! ls -alh /home/kesci/work/common/features/magic/
# ! rm /home/kesci/work/common/features/magic/debug_testMagic_localCTR.h5

total 44M
drwxr-xr-x 2 kesci root  4.0K Jul 22 11:56 .
drwxr-xr-x 6 kesci root  4.0K Jul 22 01:47 ..
-rw-r--r-- 1 kesci users 716K Jul 22 11:57 debug_trainMagic_localCTR.h5
-rw-r--r-- 1 kesci users 217K Jul 20 14:39 debug_trainMagic_queryFreq.h5
-rw-r--r-- 1 kesci users  12M Jul 22 01:37 online_testMagic_localCTR.h5
-rw-r--r-- 1 kesci users 2.4M Jul 22 01:34 online_testMagic_queryFreq.h5
-rw-r--r-- 1 kesci users  24M Jul 22 01:36 online_trainMagic_localCTR.h5
-rw-r--r-- 1 kesci users 5.0M Jul 22 01:33 online_trainMagic_queryFreq.h5


In [58]:
!ls -alh /home/kesci/work/common/features

total 20G
drwxr-xr-x 6 kesci root  4.0K Jul 22 00:01 .
drwxr-xr-x 5 kesci root  4.0K Jul 21 06:38 ..
drwxr-xr-x 2 kesci root  4.0K Jul 21 23:38 magic
-rw-r--r-- 1 kesci users 1.8K Jul 21 06:57 nd_debugConcatFeatures_name.csv
-rw-r--r-- 1 kesci users 374M Jul 21 06:57 nd_debugConcatFeatures.npy
-rw-r--r-- 1 kesci users 1.8K Jul 22 00:01 nd_onlineTrainConcatFeatures_name.csv
-rw-r--r-- 1 kesci users  18G Jul 22 00:03 nd_onlineTrainConcatFeatures.npy
drwxr-xr-x 2 kesci root  4.0K Jul 21 13:21 textMining
-rw-r--r-- 1 kesci users 7.7M Jul 21 06:57 train_labels_last_100w.npy
-rw-r--r-- 1 kesci users 382M Jul 21 23:39 train_labels_last_5000w.npy
-rw-r--r-- 1 kesci users  16M Jul 21 06:57 train_qauc_last_100w.npy
-rw-r--r-- 1 kesci users 763M Jul 21 23:40 train_qauc_last_5000w.npy
drwxr-xr-x 2 kesci root  4.0K Jul 21 23:29 vectorSpace
drwxr-xr-x 2 kesci root  4.0K Jul 21 23:21 word2vec


In [65]:
!ls -alh /home/kesci/work/outputs/test

total 153M
drwxr-xr-x 2 kesci root  4.0K Jul 22 04:12 .
drwxr-xr-x 5 kesci root  4.0K Jul 21 07:40 ..
-rw-r--r-- 1 kesci users 153M Jul 22 04:12 lgb_gbdt_nd_auc0.687899_loss0.444129_std0.002588_0722040319.npy


## 数据分析

title频次分析

In [9]:
def analysis_t_freq(source_file, names):
    tmp = pd.read_csv(source_file, names=names)
    t_counts = tmp.groupby("query_id", as_index=False)["query_title_id"].count()
    print(t_counts.describe())
    del tmp
    gc.collect()

In [11]:
analysis_t_freq("/home/kesci/work/inputs/train_last_5000w.csv", ORI_TRAIN_NAMES)

           query_id  query_title_id
count  5.045741e+06    5.045741e+06
mean   9.345222e+07    9.909347e+00
std    1.456580e+06    6.492890e+00
min    9.092935e+07    3.000000e+00
25%    9.219078e+07    4.000000e+00
50%    9.345222e+07    7.000000e+00
75%    9.471366e+07    1.700000e+01
max    9.597509e+07    2.000000e+01


In [25]:
def analysis_t_freq(source_file, names):
    tmp = pd.read_csv(source_file, names=names)
    t_counts = tmp.groupby("query_id", as_index=False)
    print(t_counts.size().values)
    del tmp
    gc.collect()

In [26]:
analysis_t_freq("/home/kesci/work/inputs/train_last_100w.csv", ORI_TRAIN_NAMES)

[ 8  4 12 ...  6  3  9]


label比例

In [22]:
def analysis_label_freq(source_file, names):
    tmp = pd.read_csv(source_file, names=names)
    label_counts = tmp["label"].value_counts()
    print(label_counts)
    total = sum(label_counts)
    print(label_counts.apply(lambda x: x/total))
    del tmp
    gc.collect()

In [24]:
analysis_label_freq("/home/kesci/work/inputs/train_last_5000w.csv", ORI_TRAIN_NAMES)

0    40885881
1     9114119
Name: label, dtype: int64
0    0.817718
1    0.182282
Name: label, dtype: float64


In [27]:
import sys, gc, csv
import numpy as np
import pandas as pd

In [34]:
def ConcatFeatures(source_files, target_dir, prefix, verbose=True):
    with Timer("load features"):
        tmp = [ReduceMemUsage(pd.read_hdf(f)) for f in source_files]
        features = pd.concat(tmp, axis=1)
        del tmp
        gc.collect()
    with Timer("deal with invalid values"):
        features = features.replace(np.inf, np.nan)
        features = features.fillna(0.0)
    if verbose:
        print("features shape:", features.shape)
        print("two rows of features:")
        print(features[:2])
    if target_dir[-1] == "/": target_dir = target_dir[:-1]
    save_file = "{}/{}_concatFeatures.npy".format(target_dir, prefix)
    save_names_file = "{}/{}_concatFeatures_name.txt".format(target_dir, prefix)
    with open(save_names_file, "w") as f:
        f.write(",".join(list(features.columns.values)))
        print("concat features names saved to {}".format(save_names_file))
    np.save(save_file, features.values)
    print("concat features saved to {}".format(save_file))
    del features
    gc.collect()
    

In [3]:
# feature_base_dir = "/home/kesci/work/common/features/"
# source_files = [
#     "textMining/debug_trainTextMining_len.h5",
#     "textMining/debug_trainTextMining_ngramSim.h5",
#     "textMining/debug_trainTextMining_editSim.h5",
#     "textMining/debug_trainTextMining_hashSim.h5",
#     "vectorSpace/debug_trainVectorSpace_tfidfVecLen.h5",
#     "vectorSpace/debug_trainVectorSpace_tfidfVecSim.h5",
#     "word2vec/debug_trainWord2vec_avgSenVecSim.h5",
#     "magic/debug_trainMagic_localCTR.h5",
#     "magic/debug_trainMagic_queryFreq.h5",
# ]
# source_files = [feature_base_dir + _ for _ in source_files]
# ConcatFeatures(source_files, "/home/kesci/work/common/features", "nd")

In [4]:
# AnalysisHDF("/home/kesci/work/common/features/textMining/online_trainTextMining_len.h5")

In [2]:
! ls -alh /home/kesci/work/common/features/textMining/
! ls -alh /home/kesci/work/common/features/vectorSpace/
! ls -alh /home/kesci/work/common/features/word2vec/
! ls -alh /home/kesci/work/common/features/magic/

total 6.3G
drwxr-xr-x 2 kesci root  4.0K Aug  4 01:54 .
drwxr-xr-x 7 kesci root  4.0K Aug  3 10:33 ..
-rw-r--r-- 1 kesci users  12M Aug  2 14:57 debug_testTextMining_editSim.h5
-rw-r--r-- 1 kesci users 2.7M Aug  2 13:50 debug_testTextMining_idfWordShare.h5
-rw-r--r-- 1 kesci users  12M Aug  2 14:13 debug_testTextMining_len.h5
-rw-r--r-- 1 kesci users 1.9M Aug  2 13:35 debug_testTextMining_normalWordShare.h5
-rw-r--r-- 1 kesci users 1.9M Aug  2 13:11 debug_testTextMining_wordShare.h5
-rw-r--r-- 1 kesci users  29M Aug  2 14:55 debug_trainTextMining_editSim.h5
-rw-r--r-- 1 kesci users  15M Jul 23 08:47 debug_trainTextMining_editSim_xp.h5
-rw-r--r-- 1 kesci users 724K Jul 20 06:54 debug_trainTextMining_hashSim.h5
-rw-r--r-- 1 kesci users 6.8M Aug  2 13:49 debug_trainTextMining_idfWordShare.h5
-rw-r--r-- 1 kesci users  29M Aug  2 14:10 debug_trainTextMining_len.h5
-rw-r--r-- 1 kesci users 6.0M Jul 22 09:26 debug_trainTextMining_len_xp.h5
-rw-r--r-- 1 kesci users  37M Jul 21 07:49 debug_trai

In [6]:
AnalysisHDF("/home/kesci/work/common/features/textMining/online_testTextMining_normalWordShare.h5")

--->file shape:
(20000000, 1)
--->2 rows of file:
   online_testTextMining_unigrams_normalWordShare_normal
0                                           0.300000    
1                                           0.666667    


In [11]:
AnalysisHDF("/home/kesci/work/common/features/word2vec/online_trainWord2vec_avgSenVecSim.h5")

--->file shape:
(50000000, 3)
--->2 rows of file:
   online_trainWord2vec_cosine_distance  \
0                              0.613462   
1                              0.751774   

   online_trainWord2vec_euclidean_distance  \
0                                12.311190   
1                                10.890883   

   online_trainWord2vec_cityblock_distance  
0                               141.610352  
1                               121.117691  


In [11]:
! ls -alh /home/kesci/work/common/features

total 33G
drwxr-xr-x 7 kesci root  4.0K Aug  3 10:33 .
drwxr-xr-x 6 kesci root  4.0K Jul 31 16:21 ..
drwxr-xr-x 2 kesci root  4.0K Aug  3 10:54 balanceSamples
-rw-r--r-- 1 kesci users 2.1K Jul 28 08:40 dj_debug_concat_features_name.csv
-rw-r--r-- 1 kesci users 451M Jul 28 08:44 dj_debug_concat_features.npy
drwxr-xr-x 2 kesci root  4.0K Jul 23 15:59 magic
-rw-r--r-- 1 kesci users 2.1K Jul 29 23:54 nd_debugConcatFeatures_name.csv
-rw-r--r-- 1 kesci users 451M Jul 29 23:54 nd_debugConcatFeatures.npy
-rw-r--r-- 1 kesci users  98M Aug  2 14:58 nd_debugTestConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 1.8K Aug  2 14:58 nd_debugTrainConcatFeaturesV1_name.csv
-rw-r--r-- 1 kesci users 244M Aug  2 14:58 nd_debugTrainConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 7.2G Jul 30 16:10 nd_onlineTestConcatFeatures.npy
-rw-r--r-- 1 kesci users 1.8G Aug  2 04:54 nd_onlineTestConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 1.8K Jul 30 15:58 nd_onlineTrainConcatFeatures_name.csv
-rw-r--r-- 1 kesci users  18G 

In [20]:
import re
def findInvalidRows(source_csv):
    invalid_count = 0
    f = open(source_csv)
    lines = f.readlines(256*1024*1024)
    count = 0
    while lines:
        for line in lines:
            count += 1
            if count <= 20:
                print(line.strip())
            if re.search(r"\t", line.strip()):
                invalid_count += 1
            if count % 1000000 == 0:
                print("count ", count)
        lines = f.readlines(256*1024*1024)
    f.close()
    print("Invalid count:", invalid_count)

In [6]:
# findInvalidRows("/home/kesci/input/bytedance/test_final_part1.csv")

In [2]:
tmp = ReadCSV1("/home/kesci/input/bytedance/test_final_part1.csv")

count: 5000000
count: 10000000
count: 15000000
count: 20000000


In [3]:
print(len(tmp))

2


In [5]:
import copy
query = copy.deepcopy(tmp[0])

In [6]:
tmp.clear()
gc.collect()

8

In [6]:
print(tmp[:10])

NameError: name 'tmp' is not defined

In [2]:
! ls -alh /home/kesci/work/common/features/word2vec

total 1.6G
drwxr-xr-x 2 kesci root  4.0K Jul 30 05:26 .
drwxr-xr-x 6 kesci root  4.0K Jul 28 14:29 ..
-rw-r--r-- 1 kesci users  14M Jul 29 23:52 debug_trainWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 264M Jul 24 17:58 online_testWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 646M Jul 24 16:06 online_trainWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 648M Jul 30 05:33 online_trainWord2vec_avgSenVecSimV1.h5


## 提交结果

In [66]:
!wget -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-07-22 04:33:49--  https://www.heywhale.com/kesci_submit
Resolving www.heywhale.com (www.heywhale.com)... 106.15.25.147
Connecting to www.heywhale.com (www.heywhale.com)|106.15.25.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6528405 (6.2M) [application/octet-stream]
Saving to: ‘kesci_submit’

kesci_submit        100%[===================>]   6.23M  20.7MB/s    in 0.3s    

2019-07-22 04:33:50 (20.7 MB/s) - ‘kesci_submit’ saved [6528405/6528405]



In [3]:
! ls -alh /home/kesci/work/outputs/test

total 306M
drwxr-xr-x 2 kesci users 4.0K Aug  2 12:15 .
drwxr-xr-x 5 kesci root  4.0K Aug  2 11:51 ..
-rw-r--r-- 1 kesci users 153M Aug  2 12:15 lgb_gbdt_nd_auc0.605735_loss0.461848_std0.001491_0802121101.npy
-rw-r--r-- 1 kesci users 153M Aug  2 12:03 lgb_gbdt_nd_auc0.605913_loss0.461650_std0.001214_0802115855.npy


In [15]:
print(tmp[:10])

[0.13107937 0.17287524 0.16051305 0.17301155 0.16706542 0.13539502
 0.1921765  0.206531   0.18293742 0.18483084]


In [2]:
# /home/kesci/work/outputs/test/lgb_gbdt_nd_auc0.688546_loss0.442005_std0.003171_0722092236.npy
res_id = "lgb_gbdt_nd_auc0.000000_loss1.144877_std0.008704_0805144356"
out_res_file = "/home/kesci/work/outputs/test/"+res_id+".npy"
test_file = "/home/kesci/work/inputs/test_final_part1.csv"

In [5]:
out_test_res = np.load(out_res_file)
test_csv = ReadCSV(test_file, names=ORI_TEST_NAMES, iterator=False)

In [6]:
res = pd.DataFrame({
    "query_id": test_csv["query_id"],
    "query_title_id": test_csv["query_title_id"],
    "label":out_test_res
})
res.to_csv("/home/kesci/work/outputs/submits/"+res_id+".csv", index=None, header=None)

In [7]:
!https_proxy="http://klab-external-proxy" ./kesci_submit -file /home/kesci/work/outputs/submits/lgb_gbdt_nd_auc0.000000_loss1.144877_std0.008704_0805144356.csv -token c0c21925b1bb5cde

Kesci Submit Tool 3.0

> 已验证Token
> 提交文件 /home/kesci/work/outputs/submits/lgb_gbdt_nd_auc0.000000_loss1.144877_std0.008704_0805144356.csv (563642.67 KiB)
> 文件已上传        
> 提交完成


## kill

In [22]:
import os
print(os.getpid())

92


In [ ]:
! kill -9 92

## 去除title\t

In [31]:
def rmt(source_csv, names, save_file):
    tmp = pd.read_csv(source_csv, names=names)
    # print(tmp[:10])
    tmp["title"] = tmp["title"].str.strip()
    # print(tmp[:10])
    tmp.to_csv(save_file, header=None, index=None)
    del tmp
    gc.collect()
# rmt("/home/kesci/input/bytedance/test_final_part1.csv", ORI_TEST_NAMES, "/home/kesci/work/inputs/test_final_part1.csv")


## 训练tfidf

In [ ]:
import pickle, sys, gc, time, os
import pandas as pd
import numpy as np
import csv
sys.path.append("./common")
model_save_dir = "/home/kesci/work/common/util_models/"
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TEST_NAMES, \
ProcessChunk, AnalysisCSV, ReadCSV1
OFFLINE = True
train_name = ORI_TRAIN_NAMES
if OFFLINE:
    train_file = "/home/kesci/work/inputs/train_flast_100w.csv"
    test_file = "/home/kesci/work/inputs/train_flast_40w.csv"
    test_name = ORI_TRAIN_NAMES
else:
    train_file = "/home/kesci/work/inputs/train_first_5000w.csv"
    test_file = "/home/kesci/work/inputs/test_final_part1.csv"
    test_name = ORI_TEST_NAMES
! ls -alh /home/kesci/work/common/util_models/
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
ngram_range, ngram_level = (1, 3), "word"
suffix = "allUnique"
if OFFLINE:
    mode = "debug"
else:
    mode = "online"
tfidf_save_file = "{}tfidf_{}_ngram-{}{}_{}_{}.bin".format(model_save_dir, ngram_level, 
    ngram_range[0], ngram_range[1], mode, suffix)
regen_tfidf_model = True
print(tfidf_save_file)
def train_tfidf(save_file):
    word_tfidf = TfidfVectorizer(norm="l2",  # 'l2 norm' can cal similarty
                            strip_accents="unicode",
                            analyzer=ngram_level,
                            ngram_range=ngram_range,
                            use_idf=True,
                            smooth_idf=True,
                            sublinear_tf=True, min_df=2, max_df=0.9)
    with Timer("prepare corpus"):
        df_train = ReadCSV(train_file, names=train_name, iterator=False)
        df_test = ReadCSV(test_file, names=test_name,  iterator=False)
        df_data = pd.concat([df_train[['query', 'title']], df_test[['query', 'title']]], axis=0)
        del df_train, df_test; gc.collect()
        corpus =  pd.Series(
            df_data['query'].unique().tolist() +
            df_data['title'].unique().tolist()
        ).astype(str)
        print(corpus.shape)
    with Timer("train tfidf"):
        word_tfidf.fit(corpus)
    with Timer("save tfidf"):
        with open(save_file, "wb") as f:
            pickle.dump(word_tfidf, f)
            print("Model dumped to {}".format(save_file))
    return None
# 提取训练数据特征
if regen_tfidf_model:
    train_tfidf(tfidf_save_file)

total 22G
drwxr-xr-x 2 kesci root  4.0K Jul 31 23:25 .
drwxr-xr-x 6 kesci root  4.0K Jul 31 16:21 ..
-rw-r--r-- 1 kesci root   19K Jul 29 13:56 DeepFM.py
-rw-r--r-- 1 kesci users 249M Jul 22 03:01 tfidf_word_ngram-13-100w.bin
-rw-r--r-- 1 kesci users 6.2G Jul 23 17:48 tfidf_word_ngram-13.bin
-rw-r--r-- 1 kesci users 929M Jul 31 12:26 tfidf_word_ngram-13_debug_allUnique.bin
-rw-r--r-- 1 kesci users 702M Jul 31 12:36 tfidf_word_ngram-13_debug_justTrain.bin
-rw-r--r-- 1 kesci users 929M Jul 31 09:26 tfidf_word_ngram-13_debug_noUnique.bin
-rw-r--r-- 1 kesci users 1.1G Jul 31 12:53 tfidf_word_ngram-13_debug_queryConcatTitle.bin
-rw-r--r-- 1 kesci users 929M Jul 31 09:30 tfidf_word_ngram-13_debug_queryUnique.bin
-rw-r--r-- 1 kesci users 7.6G Jul 31 16:11 tfidf_word_ngram-13_online_allUnique.bin
-rw-r--r-- 1 kesci users  11M Jul 31 13:29 word2vec_debug_allUnique.kv
-rw-r--r-- 1 kesci users  70M Jul 31 13:29 word2vec_debug_allUnique.kv.vectors.npy
-rw-r--r-- 1 kesci users  11M Jul 31 13:26 wor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



----->finished 'train tfidf' block, time used:1.28s.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-3b1c6b553e94>", line 59, in <module>
    train_tfidf(tfidf_save_file)
  File "<ipython-input-2-3b1c6b553e94>", line 51, in train_tfidf
    word_tfidf.fit(corpus)
  File "/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 1584, in fit
    X = super(TfidfVectorizer, self).fit_transform(raw_documents)
  File "/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 1032, in fit_transform
    self.fixed_vocabulary_)
  File "/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 942, in _count_vocab
    for feature in analyze(doc):
  File "/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py", line 328, in <lambda>
    token

KeyboardInterrupt: 

In [2]:
!head /home/kesci/work/inputs/word2vec_sentences_flast_100w.txt

844 10482 6804 8411
844 6026 15 53 100
51685 402
1913 1278 15 2215 1039
4311 456 15 321608 95
53058 31 50989 3028
236 127 1364 55882 2824
236 277 3705 2848 10035
236 160931 10592
236 68629 5975 100 29816 87051


In [3]:
!head /home/kesci/work/inputs/train_flast_100w.csv

4847148,844 10482 6804 8411,3,2833 83785 8411,0	
4847148,844 10482 6804 8411,4,7469 59 15 844 36 6804 8411,0	
4847148,844 10482 6804 8411,5,47 628 6804 8411 698 53 27 4473 542 3922 2432 27 190 158 1235 64,1	
4847149,844 6026 15 53 100,1,47 22 9421 118 6026 15 53 27 48 294 382,0	
4847149,844 6026 15 53 100,2,47 22 7 21 120 45944 27 1568 188 292 27 106 7 1158 40 77 4812 27 53 322,0	
4847149,844 6026 15 53 100,3,47 22 745 902 14336 6026 15 322 53,0	
4847149,844 6026 15 53 100,4,170 844 171 22 1145 312 1272 15 26085 53 100 27 344 120 11,1	
4847149,844 6026 15 53 100,5,20868 2761 15 6026 15 53 100 27 45 120 31 27 1372 5 38 1373 11,0	
4847149,844 6026 15 53 100,6,7 31 3879 6026 15 826 27 47 628 11236 698 53 27 42 47 628 45944 15 11181 250,1	
4847149,844 6026 15 53 100,7,184 1312 25 4456 12 6026 27 255 640 6057 27 7417 937 3573 27 7 241 41 5 29 4126,0	


In [4]:
!head /home/kesci/work/inputs/word2vec_sentences_flast_5000w.txt

1427 4337 2241 38448 24645 96569 2241 71 357637 78 448707 70
1427 4337 2241 38448 2241 28562 3250 4337 34 40280
1427 4337 2241 54 27846 15 237 200 2241 59 786 4337
1427 4337 2241 756 4337 2241 39645 399 2 22 408 27 22 55 408 559 399 37
1427 4337 2241 42178 4337 2241 91 2345 36 93561 128362 195219 272650 6803 89200 155887 66422
1427 4337 2241 38448 24645 96569 2241 71 684027 78 671264 70
1427 20 496 13 64441 1427 360 1835 20 11132 9 135583 496 682 796 19991 23797
1427 20 496 13 64441 1427 5331 43 20 13 64441 496 37 167 32859 36 51 1985 496 625 49 1985 101893 27 4418 14685
1427 20 496 13 64441 1427 2815 20132 11 20 496 117 824 23791 27 141 36 153 9 2113 95
1427 20 496 13 64441 1427 20 272 518 20132 496 37 209 42 13 64441 496 1327 33 12 11


In [5]:
!head /home/kesci/work/inputs/train_first_5000w.csv

1,1427 4337 2241,1,38448 24645 96569 2241 71 357637 78 448707 70,1	
1,1427 4337 2241,2,38448 2241 28562 3250 4337 34 40280,1	
1,1427 4337 2241,3,54 27846 15 237 200 2241 59 786 4337,0	
1,1427 4337 2241,4,756 4337 2241 39645 399 2 22 408 27 22 55 408 559 399 37,0	
1,1427 4337 2241,5,42178 4337 2241 91 2345 36 93561 128362 195219 272650 6803 89200 155887 66422,0	
1,1427 4337 2241,6,38448 24645 96569 2241 71 684027 78 671264 70,1	
2,1427 20 496 13 64441 1427,1,360 1835 20 11132 9 135583 496 682 796 19991 23797,0	
2,1427 20 496 13 64441 1427,2,5331 43 20 13 64441 496 37 167 32859 36 51 1985 496 625 49 1985 101893 27 4418 14685,0	
2,1427 20 496 13 64441 1427,3,2815 20132 11 20 496 117 824 23791 27 141 36 153 9 2113 95,0	
2,1427 20 496 13 64441 1427,4,20 272 518 20132 496 37 209 42 13 64441 496 1327 33 12 11,1	


## 提取特征

In [2]:
import sys, os, gc, pickle
import numpy as np
import pandas as pd
import scipy
# import modin.pandas as pd
sys.path.append("./common")
sys.path.append("./dongdong")
sys.path.append("./dongdong/utils")
from np_utils import try_divide
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TEST_NAMES, AnalysisCSV, \
FuncMap2, FuncMap1, AnalysisHDF, ExtractFeature1
import dist_utils, ngram_utils
base_feature_save_dir = "/home/kesci/work/common/features/"
tfidf_model_file = "/home/kesci/work/common/util_models/tfidf_word_ngram-13_online_allUnique.bin"
word2vec_kv_model_file = "/home/kesci/work/common/util_models/word2vec_online_concatHorizontal.kv"
word2vec_embed_size = 100
# True: 离线训练(100w)
# False: 上线训练(5000w)
OFFLINE = True
train_name = ORI_TRAIN_NAMES
if OFFLINE:
    base_prefix = "debug_"
    train_file = "/home/kesci/work/inputs/train_flast_100w.csv"
    test_file = "/home/kesci/work/inputs/train_flast_40w.csv"
    CHUNK_SIZE = 500000
    test_name = ORI_TRAIN_NAMES
else:
    base_prefix = "online_"
    train_file = "/home/kesci/work/inputs/train_first_5000w.csv"
    test_file = "/home/kesci/work/inputs/test_final_part1.csv"
    CHUNK_SIZE = 50000000 # 256*1024*1024
    test_name = ORI_TEST_NAMES
print("OFFLINE:", OFFLINE)
print(train_file)
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
feature_save_dir = base_feature_save_dir + "vectorSpace/"
train_feature_prefix = base_prefix + "trainVectorSpace"
test_feature_prefix = base_prefix + "testVectorSpace"
print(tfidf_model_file)
! ls -alh /home/kesci/work/common/features/vectorSpace
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
from sklearn.metrics.pairwise import *
regen_tfidf_vec_sim = True
feature_name = "tfidfVecFeat"
print(tfidf_model_file)
dis_arr = [
    paired_cosine_distances,
    paired_euclidean_distances,
    paired_manhattan_distances
]
def process_tfidf_vec_feat(df, save_dir, prefix, feature_name):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec = tfidf_model.transform(df["query"])
        t_vec = tfidf_model.transform(df["title"])
    with Timer("cal tfidf vec value"):
        q_vec_sum = np.array(q_vec.sum(1)).squeeze()
        t_vec_sum = np.array(t_vec.sum(1)).squeeze()
        df['{}_{}_QSum'.format(prefix, feature_name)] = q_vec_sum
        df['{}_{}_TSum'.format(prefix, feature_name)] = t_vec_sum
        del q_vec_sum, t_vec_sum
        gc.collect()
    with Timer("cal tfidf vec sim"):
        for _ in dis_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}_{}".format(prefix, feature_name, name)] = _(q_vec, t_vec)
        del q_vec, t_vec, tfidf_model
        gc.collect()
    return df
# 提取训练数据特征
# if regen_tfidf_vec_sim:
#     ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
#                   process_func=process_tfidf_vec_feat, names=train_name, process_chunkly=False, 
#                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
#                   drop_last_cols=['query', 'title'])
# 提取训练数据特征
if regen_tfidf_vec_sim:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
                   process_func=process_tfidf_vec_feat, names=test_name, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id'], 
                   drop_last_cols=['query', 'title'])

OFFLINE: True
/home/kesci/work/inputs/train_flast_100w.csv
/home/kesci/work/common/util_models/tfidf_word_ngram-13_online_allUnique.bin
total 12G
drwxr-xr-x 2 kesci root  4.0K Aug  1 12:01 .
drwxr-xr-x 6 kesci root  4.0K Jul 31 12:06 ..
-rw-r--r-- 1 kesci users  13M Aug  1 12:01 debug_testVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users  33M Aug  1 11:52 debug_trainVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users  52M Jul 21 08:02 debug_trainVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users  52M Jul 29 23:47 debug_trainVectorSpace_tfidfVecLenV1.h5
-rw-r--r-- 1 kesci users  19M Jul 22 11:26 debug_trainVectorSpace_tfidfVecSim.h5
-rw-r--r-- 1 kesci users  19M Jul 29 23:38 debug_trainVectorSpace_tfidfVecSimV1.h5
-rw-r--r-- 1 kesci users  20M Jul 22 03:09 debug_trainVectorSpace_tfidfVecSim_xp.h5
-rw-r--r-- 1 kesci users 631M Aug  1 07:40 online_testVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users 980M Jul 24 04:13 online_testVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users 

In [6]:
AnalysisHDF("/home/kesci/work/common/features/vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5")

--->file shape:
(400000, 5)
--->2 rows of file:
   debug_testVectorSpace_tfidfVecFeat_QSum  \
0                                 1.913374   
1                                 1.913374   

   debug_testVectorSpace_tfidfVecFeat_TSum  \
0                                 5.618418   
1                                 4.642403   

   debug_testVectorSpace_tfidfVecFeat_paired_cosine_distances  \
0                                           0.704403            
1                                           0.635939            

   debug_testVectorSpace_tfidfVecFeat_paired_euclidean_distances  \
0                                           1.186931               
1                                           1.127776               

   debug_testVectorSpace_tfidfVecFeat_paired_manhattan_distances  
0                                           6.473185              
1                                           5.251984              


In [5]:
data = pd.read_hdf("/home/kesci/work/common/features/vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5")
data.drop(["label"], axis=1, inplace=True)
data.to_hdf(
    "/home/kesci/work/common/features/vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5", 
    mode="w", key="df", complib="bzip2", complevel=9)

In [11]:
data = pd.read_hdf("/home/kesci/work/common/features/vectorSpace/online_testVectorSpace_tfidfVecFeat.h5")

In [10]:
data[1:].to_hdf(
    "/home/kesci/work/common/features/vectorSpace/online_testVectorSpace_tfidfVecFeat.h5", 
    mode="w", key="df", complib="bzip2", complevel=9)

In [13]:
print(data.insert()

(19999999, 5)
